In [ ]:
import nltk
import numpy as np
from scipy.io import loadmat
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from scipy import stats
from sklearn.decomposition import PCA
from IPython.core.debugger import set_trace
import pickle
import os
from joblib import Parallel, delayed

results_save_dir = "predictions"
features_dir = "features"
sub_data = "sub_space_data/"

if not os.path.exists(results_save_dir):
    os.mkdir(results_save_dir)

In [ ]:
# Functions to estimate cost for each lambda, by voxel:
from __future__ import division                                              

from numpy.linalg import inv, svd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, RidgeCV
import time 
from scipy.stats import zscore


def corr(X,Y):
    return np.mean(zscore(X)*zscore(Y),0)

def R2(Pred,Real):
    SSres = np.mean((Real-Pred)**2,0)
    SStot = np.var(Real,0)
    return np.nan_to_num(1-SSres/SStot)

def R2r(Pred,Real):
    R2rs = R2(Pred,Real)
    ind_neg = R2rs<0
    R2rs = np.abs(R2rs)
    R2rs = np.sqrt(R2rs)
    R2rs[ind_neg] *= - 1
    return R2rs

def ridge(X,Y,lmbda):
    return np.dot(inv(X.T.dot(X)+lmbda*np.eye(X.shape[1])),X.T.dot(Y))

def ridge_by_lambda(X, Y, Xval, Yval, lambdas=np.array([0.1,1,10,100,1000])):
    error = np.zeros((lambdas.shape[0],Y.shape[1]))
    for idx,lmbda in enumerate(lambdas):
        weights = ridge(X,Y,lmbda)
        error[idx] = 1 - R2(np.dot(Xval,weights),Yval)
    return error

def ridge_sk(X,Y,lmbda):
    rd = Ridge(alpha = lmbda)
    rd.fit(X,Y)
    return rd.coef_.T

def ridgeCV_sk(X,Y,lmbdas):
    rd = RidgeCV(alphas = lmbdas,solver = 'svd')
    rd.fit(X,Y)
    return rd.coef_.T

def ridge_by_lambda_sk(X, Y, Xval, Yval, lambdas=np.array([0.1,1,10,100,1000])):
    error = np.zeros((lambdas.shape[0],Y.shape[1]))
    for idx,lmbda in enumerate(lambdas):
        weights = ridge_sk(X,Y,lmbda)
        error[idx] = 1 -  R2(np.dot(Xval,weights),Yval)
    return error

def ridge_svd(X,Y,lmbda):
    U, s, Vt = svd(X, full_matrices=False)
    d = s / (s** 2 + lmbda)
    return np.dot(Vt,np.diag(d).dot(U.T.dot(Y)))

def ridge_by_lambda_svd(X, Y, Xval, Yval, lambdas=np.array([0.1,1,10,100,1000])):
    error = np.zeros((lambdas.shape[0],Y.shape[1]))
    U, s, Vt = svd(X, full_matrices=False)
    for idx,lmbda in enumerate(lambdas):
        d = s / (s** 2 + lmbda)
        weights = np.dot(Vt,np.diag(d).dot(U.T.dot(Y)))
        error[idx] = 1 - R2(np.dot(Xval,weights),Yval)
    return error


def kernel_ridge(X,Y,lmbda):
    return np.dot(X.T.dot(inv(X.dot(X.T)+lmbda*np.eye(X.shape[0]))),Y)

def kernel_ridge_by_lambda(X, Y, Xval, Yval, lambdas=np.array([0.1,1,10,100,1000])):
    error = np.zeros((lambdas.shape[0],Y.shape[1]))
    for idx,lmbda in enumerate(lambdas):
        weights = kernel_ridge(X,Y,lmbda)
        error[idx] = 1 - R2(np.dot(Xval,weights),Yval)
    return error


def kernel_ridge_svd(X,Y,lmbda):
    U, s, Vt = svd(X.T, full_matrices=False)
    d = s / (s** 2 + lmbda)
    return np.dot(np.dot(U,np.diag(d).dot(Vt)),Y)

def kernel_ridge_by_lambda_svd(X, Y, Xval, Yval, lambdas=np.array([0.1,1,10,100,1000])):
    error = np.zeros((lambdas.shape[0],Y.shape[1]))
    U, s, Vt = svd(X.T, full_matrices=False)
    for idx,lmbda in enumerate(lambdas):
        d = s / (s** 2 + lmbda)
        weights = np.dot(np.dot(U,np.diag(d).dot(Vt)),Y)
        error[idx] = 1 - R2(np.dot(Xval,weights),Yval)
    return error


def cross_val_ridge(train_features,train_data, n_splits = 10, 
                    lambdas = np.array([10**i for i in range(-6,10)]),
                    method = 'plain',
                    do_plot = False):
    
    ridge_1 = dict(plain = ridge_by_lambda,
                   svd = ridge_by_lambda_svd,
                   kernel_ridge = kernel_ridge_by_lambda,
                   kernel_ridge_svd = kernel_ridge_by_lambda_svd,
                   ridge_sk = ridge_by_lambda_sk)[method]
    ridge_2 = dict(plain = ridge,
                   svd = ridge_svd,
                   kernel_ridge = kernel_ridge,
                   kernel_ridge_svd = kernel_ridge_svd,
                   ridge_sk = ridge_sk)[method]
    
    n_voxels = train_data.shape[1]
    nL = lambdas.shape[0]
    r_cv = np.zeros((nL, train_data.shape[1]))

    kf = KFold(n_splits=n_splits)
    start_t = time.time()
    for icv, (trn, val) in enumerate(kf.split(train_data)):
#         print('ntrain = {}'.format(train_features[trn].shape[0]))
        cost = ridge_1(train_features[trn],train_data[trn],
                               train_features[val],train_data[val], 
                               lambdas=lambdas)
        if do_plot:
            import matplotlib.pyplot as plt
            plt.figure()
            plt.imshow(cost,aspect = 'auto')
        r_cv += cost
#         if icv%3 ==0:
#             print(icv)
#         print('average iteration length {}'.format((time.time()-start_t)/(icv+1)))
    if do_plot:
        plt.figure()
        plt.imshow(r_cv,aspect='auto',cmap = 'RdBu_r');

    argmin_lambda = np.argmin(r_cv,axis = 0)
    weights = np.zeros((train_features.shape[1],train_data.shape[1]))
    for idx_lambda in range(lambdas.shape[0]): # this is much faster than iterating over voxels!
        idx_vox = argmin_lambda == idx_lambda
        weights[:,idx_vox] = ridge_2(train_features, train_data[:,idx_vox],lambdas[idx_lambda])
    if do_plot:
        plt.figure()
        plt.imshow(weights,aspect='auto',cmap = 'RdBu_r',vmin = -0.5,vmax = 0.5);

    return weights, np.array([lambdas[i] for i in argmin_lambda])




def GCV_ridge(train_features,train_data,lambdas = np.array([10**i for i in range(-6,10)])):
    
    n_lambdas = lambdas.shape[0]
    n_voxels = train_data.shape[1]
    n_time = train_data.shape[0]
    n_p = train_features.shape[1]

    CVerr = np.zeros((n_lambdas, n_voxels))

    # % If we do an eigendecomp first we can quickly compute the inverse for many different values
    # % of lambda. SVD uses X = UDV' form.
    # % First compute K0 = (X'X + lambda*I) where lambda = 0.
    #K0 = np.dot(train_features,train_features.T)
    print('Running svd',)
    start_time = time.time()
    [U,D,Vt] = svd(train_features,full_matrices=False)
    V = Vt.T
    print(U.shape,D.shape,Vt.shape)
    print('svd time: {}'.format(time.time() - start_time))

    for i,regularizationParam in enumerate(lambdas):
        regularizationParam = lambdas[i]
        print('CVLoop: Testing regularization param: {}'.format(regularizationParam))

        #Now we can obtain Kinv for any lambda doing Kinv = V * (D + lambda*I)^-1 U'
        dlambda = D**2 + np.eye(n_p)*regularizationParam
        dlambdaInv = np.diag(D / np.diag(dlambda))
        KlambdaInv = V.dot(dlambdaInv).dot(U.T)
        
        # Compute S matrix of Hastie Trick  H = X(XT X + lambdaI)-1XT
        S = np.dot(U, np.diag(D * np.diag(dlambdaInv))).dot(U.T)
        denum = 1-np.trace(S)/n_time
        
        # Solve for weight matrix so we can compute residual
        weightMatrix = KlambdaInv.dot(train_data);


#         Snorm = np.tile(1 - np.diag(S) , (n_voxels, 1)).T
        YdiffMat = (train_data - (train_features.dot(weightMatrix)));
        YdiffMat = YdiffMat / denum;
        CVerr[i,:] = (1/n_time)*np.sum(YdiffMat * YdiffMat,0);


    # try using min of avg err
    minerrIndex = np.argmin(CVerr,axis = 0);
    r=np.zeros((n_voxels));

    for nPar,regularizationParam in enumerate(lambdas):
        ind = np.where(minerrIndex==nPar)[0];
        if len(ind)>0:
            r[ind] = regularizationParam;
            print('{}% of outputs with regularization param: {}'.format(int(len(ind)/n_voxels*100),
                                                                        regularizationParam))
            # got good param, now obtain weights
            dlambda = D**2 + np.eye(n_p)*regularizationParam
            dlambdaInv = np.diag(D / np.diag(dlambda))
            KlambdaInv = V.dot(dlambdaInv).dot(U.T)

            weightMatrix[:,ind] = KlambdaInv.dot(train_data[:,ind]);


    return weightMatrix, r

def sub_to_subjectnum(sub):
    for i in range(len(all_subjects)):
        if all_subjects[i] == sub:
            return i+1
    return 0

In [ ]:
def eval(feature_name, save_regressed_y = False):    
    print(feature_name)
    np.random.seed(9)
    kf = KFold(n_splits=4)
    
    # read features
    features = np.load(os.path.join(features_dir,"{}.npy".format(feature_name)))
    features = features.reshape(features.shape[0], -1)
    features = [list(features[i]) for i in range(len(features))]
    print(len(features))
    
    # align features with fMRI data and create final
    history = 4
    featlen = len(features[0])
    x = []

    x = features[:1303] + [[0.0]*featlen] + features[1303:2654] + [[0.0]*featlen] \
            + features[2654:3713] + [[0.0]*featlen] + features[3713:] + [[0.0]*featlen]

    for j in range(history*4):
        x = [[0.0]*featlen] + x

    features_tr = []
    k = history
    while(k<len(x)/4):
        if len(features_tr)<=(k-history):
            features_tr.append([])
        for o in range(history,0,-1):
            features_tr[k-history] = features_tr[k-history] + np.sum([j for j in x[(k-o)*4:((k-o+1)*4)]],axis=0).tolist()
        k += 1

    features_tr = features_tr[:325] + features_tr[326:663] + features_tr[664:928] + features_tr[929:-1]
    features_tr = np.array(features_tr)

    print(features_tr.shape)

    save_dir = os.path.join(results_save_dir, feature_name)

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    for sub in all_subjects:
        print("Subject " + sub)
        if os.path.exists(os.path.join(save_dir, sub + "_r2s.npy")):
            continue
        N_s = sub_to_subjectnum(sub)
        data = np.load(sub_data + sub + ".npy")
        print(data.shape)
        print(features_tr.shape)

        r2s = np.zeros(data.shape[1])   
        corr = np.zeros((data.shape[1],2))

        split_num = 0

        all_preds = []
        all_reals = []

        for train_index, test_index in kf.split(data):
            # remove 5 TRs which either precede or follow the TRs in the test set
            train_index_without_overlap = train_index
            for rem_val in range(test_index[0] - 5, test_index[0], 1):
                train_index_without_overlap = train_index_without_overlap[train_index_without_overlap != rem_val]
                
            for rem_val in range(test_index[-1] + 1, test_index[-1] + 6, 1):
                train_index_without_overlap = train_index_without_overlap[train_index_without_overlap != rem_val]
            
            x_train, x_test = features_tr[train_index_without_overlap], features_tr[test_index]
            y_train, y_test = data[train_index_without_overlap], data[test_index]
            
            x_train = stats.zscore(x_train,axis=0)
            x_train = np.nan_to_num(x_train)

            x_test = stats.zscore(x_test,axis=0)
            x_test = np.nan_to_num(x_test)

            y_train = stats.zscore(y_train,axis=0)
            y_train = np.nan_to_num(y_train)

            y_test = stats.zscore(y_test,axis=0) 
            y_test = np.nan_to_num(y_test)
            
            print(x_train.shape)
            print(y_train.shape)
            print(x_test.shape)
            print(y_test.shape)

            weights, lbda = cross_val_ridge(x_train,y_train)        
            y_pred = np.dot(x_test,weights)

            np.save(os.path.join(save_dir, "{}_y_pred_{}".format(sub, split_num)),y_pred)
            np.save(os.path.join(save_dir, "{}_y_test_{}".format(sub, split_num)),y_test)
            np.save(os.path.join(save_dir, "{}_weights_{}".format(sub, split_num)),weights)
            np.save(os.path.join(save_dir, "{}_lbda_{}".format(sub, split_num)),lbda)
            
            if save_regressed_y:
                y_pred_train = np.dot(x_train,weights)
                np.save(os.path.join(save_dir, "{}_y_regress_test_{}".format(sub, split_num)),y_test - y_pred)
                np.save(os.path.join(save_dir, "{}_y_regress_train_{}".format(sub, split_num)),y_train - y_pred_train)

            split_num += 1

            all_reals.append(y_test)
            all_preds.append(y_pred)

        all_reals = np.vstack(all_reals)
        all_preds = np.vstack(all_preds)

        r2s = r2_score(all_reals,all_preds,multioutput="raw_values")

        for i in range(all_reals.shape[1]):
            if np.nonzero(all_reals[:,i])[0].size > 0:
                corr[i] = stats.pearsonr(all_reals[:,i],all_preds[:,i])
            else:
                r2s[i] = 0
                corr[i][1] = 1

        print(np.max(r2s))

        np.save(os.path.join(save_dir, sub + "_r2s"),r2s)
        np.save(os.path.join(save_dir, sub + "_corr"),corr)

In [ ]:
num_contrege_sets_per_space = 5
contrege_sets = ["contrege_comp", "contrege_incomp", "incontrege"]

all_features = ["punct_final", "node_count_punct", "syntactic_surprisal_punct", \
                "word_frequency_punct", "word_length_punct", "all_complexity_metrics_punct", \
                "pos_dep_tags_all_complexity_metrics"]
for set_name in contrege_sets:
    for i in range(num_contrege_sets_per_space):
        all_features.append("{}_set_{}_pos_dep_tags_all_complexity_metrics".format(set_name, i))
        if set_name == "contrege_incomp":
            all_features.append("bert_PCA_dims_15_{}_set_{}_pos_dep_tags_all_complexity_metrics".format(set_name, i))

print(all_features)

all_subjects = ["F","G","H","I","J","K","L","M","N"]

for feat in all_features:
    eval(feat)